In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import os

In [ ]:
import pandas as pd

file_path = "/content/CET_Cutoff.xlsx"
df = pd.read_excel(file_path)

print(df.head())

   college_id branch_code               branch_name  \
0        4123   412300210  Aeronautical Engineering   
1        4123   412300210  Aeronautical Engineering   
2        4123   412300210  Aeronautical Engineering   
3        4123   412300210  Aeronautical Engineering   
4        4123   412300210  Aeronautical Engineering   

                                          stage_type category   rank  \
0  Home University Seats Allotted to Home Univers...   GOPENH  59027   
1  Home University Seats Allotted to Home Univers...   LOPENH  50662   
2  Other Than Home University Seats Allotted to O...   GOPENO  38739   
3  Other Than Home University Seats Allotted to O...   LOPENO  32586   
4                                        State Level       MI  42483   

   percentile  year  round branch_category  \
0   66.024812  2022      1          CETAE1   
1   71.447047  2022      1          CETAE1   
2   78.665707  2022      1          CETAE1   
3   82.406958  2022      1          CETAE1   
4   76

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import os
from google.colab import files

if not os.path.exists("/content/CET_Cutoff.xlsx"):
    print("Please upload the file: CET_Cutoff.xlsx")
    uploaded = files.upload()  # Opens file upload dialog



In [ ]:
df = pd.read_excel(file_path)

print("📌 Column Names in Dataset:", df.columns.tolist())

📌 Column Names in Dataset: ['college_id', 'branch_code', 'branch_name', 'stage_type', 'category', 'rank', 'percentile', 'year', 'round', 'branch_category', 'college_name', 'status', 'university', 'religious_minority', 'linguistic_minority']


In [ ]:
df = df.dropna()

df = df[df["percentile"].notnull()]

label_encoder = LabelEncoder()
df['college_encoded'] = label_encoder.fit_transform(df['college_name'])

In [ ]:
if "percentile" not in df.columns:
    raise ValueError("Error: The dataset does not have a 'percentile' column! Check column names.")

df["percentile"] = pd.to_numeric(df["percentile"], errors='coerce')  # Convert to numeric (handles float)
df = df.dropna(subset=["percentile"])  # Remove NaN values

In [ ]:
import pandas as pd

file_path = "/content/CET_Cutoff.xlsx"
df = pd.read_excel(file_path)

print("🔍 First 10 values in the 'percentile' column before cleaning:\n", df["percentile"].head(10))

print("\n🛑 Count of NaN values in 'percentile':", df["percentile"].isna().sum())

print("\n🔍 Data types in the dataset:\n", df.dtypes)

🔍 First 10 values in the 'percentile' column before cleaning:
 0    66.024812
1    71.447047
2    78.665707
3    82.406958
4    76.567514
5    57.063856
6    45.271811
7    53.730279
8    50.561478
9    52.230104
Name: percentile, dtype: float64

🛑 Count of NaN values in 'percentile': 0

🔍 Data types in the dataset:
 college_id               int64
branch_code             object
branch_name             object
stage_type              object
category                object
rank                     int64
percentile             float64
year                     int64
round                    int64
branch_category         object
college_name            object
status                  object
university              object
religious_minority      object
linguistic_minority     object
dtype: object


In [ ]:
df["percentile"] = pd.to_numeric(df["percentile"], errors="coerce")

df = df.dropna(subset=["percentile"])

if df.empty:
    raise ValueError("❌ ERROR: No valid 'percentile' values found after cleaning! Check if the column contains numeric values.")

In [ ]:
df["percentile"] = pd.to_numeric(df["percentile"], errors="coerce")
df = df.dropna(subset=["percentile"])

In [ ]:
label_encoder = LabelEncoder()
df['college_encoded'] = label_encoder.fit_transform(df['college_name'])

X = df[['percentile']]
y = df['college_encoded']

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_test = lgb.Dataset(X_test, label=y_test, reference=lgb_train)

In [ ]:
params = {
    'objective': 'multiclass',
    'num_class': len(df['college_name'].unique()),
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbosity': -1
}


In [ ]:
model = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=[lgb_test])

In [ ]:
from scipy.special import softmax
import numpy as np

user_percentile = float(input("Enter your CET percentile: "))

user_input_scaled = scaler.transform([[user_percentile]])

pred_probs = model.predict(user_input_scaled)[0]

pred_probs = softmax(pred_probs)

top_indices = np.argsort(pred_probs)[::-1]
top_colleges = [(label_encoder.inverse_transform([i])[0], pred_probs[i] * 100) for i in top_indices[:10]]

max_prob = 90
min_prob = 50
num_colleges = len(top_colleges)

adjusted_colleges = []
for rank, (college, prob) in enumerate(top_colleges):
    adjusted_prob = max_prob - (rank * (max_prob - min_prob) / (num_colleges - 1))
    adjusted_colleges.append((college, adjusted_prob))

print("\n Top predicted colleges based on your percentile:")
for i, (college, prob) in enumerate(adjusted_colleges, 1):
    print(f"{i}. {college} ({prob:.2f}%)")

Enter your CET percentile: 89

🎓 Top predicted colleges based on your percentile:
1. Yeshwantrao Chavan College of Engineering,Wanadongri, Nagpur (90.00%)
2. Nutan Maharashtra Vidya Prasarak Mandal, Nutan Maharashtra Institute of Engineering &Technology, Talegaon station, Pune (85.56%)
3. Sinhgad Technical Education Society, Sinhgad Institute of Technology and Science, Narhe (Ambegaon) (81.11%)
4. Terna Engineering College, Nerul, Navi Mumbai (76.67%)
5. Progressive Education Society's Modern College of Engineering, Pune (72.22%)
6. N.Y.S.S.'s Datta Meghe College of Engineering, Airoli, Navi Mumbai (67.78%)
7. Manjara Charitable Trust's Rajiv Gandhi Institute of Technology, Mumbai (63.33%)
8. JSPM'S Jaywantrao Sawant College of Engineering,Pune (58.89%)
9. Shri Guru Gobind Singhji Institute of Engineering and Technology, Nanded (54.44%)
10. MIT Academy of Engineering,Alandi, Pune (50.00%)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
